In [1]:
import re
from pathlib import Path

import duckdb
import pandas as pd
from IPython.display import display, HTML

# METRYKI W MSL SĄ ŹLE JEŚLI BYŁA WIĘCEJ NIŻ JEDNA EPOKA!!!!! PRZEPISAĆ RĘCZNIE

# TODO SPRAWDZIĆ ZMIANĘ W DETEKCJI

In [2]:
RESULTS_PATH = Path("misc")
list(RESULTS_PATH.glob("*"))

[PosixPath('misc/results_split_train.csv'),
 PosixPath('misc/results_floodnet_initial_msl.csv'),
 PosixPath('misc/results_floodnet_finetune.csv'),
 PosixPath('misc/results_single_double.csv'),
 PosixPath('misc/results_split_msl.csv'),
 PosixPath('misc/results_split_test.csv')]

In [3]:
(results_initial_msl := pd.read_csv(RESULTS_PATH / "results_floodnet_initial_msl.csv").sort_index())
# incomplete run t_floodnet_msl_baseline_singlebranch
# results_initial_msl = results_initial_msl[results_initial_msl.Created != "2024-12-23T23:32:48.000Z"]
# duplcate initial run t_initial_floodnet_baseline_singlebranch_flood
results_initial_msl = results_initial_msl[results_initial_msl.Created != "2024-12-26T17:30:17.000Z"]
results_initial_msl = results_initial_msl[results_initial_msl.Created != "2024-12-27T19:44:01.000Z"]


results_initial_msl

,Name,State,Created,Runtime,epoch,challenge_score_target,best_challenge_score_target,best_challenge_score_target_epoch,f1_target,f1_loc_target,...,f1_target_3,f1_target_4,challenge_score_source,f1_loc_source,f1_source_class,f1_source_0,f1_source_1,f1_source_2,f1_source_3,f1_source_4
1,t_initial_floodnet_baseline_singlebranch_flood,finished,2024-12-27T19:42:57.000Z,62,0,0.559746,0.559746,0.0,0.580215,0.651327,...,NaN,NaN,0.599780,0.731908,0.543154,0.983879,0.684317,0.497322,0.664632,0.311290
2,t_floodnet_msl_farseg_single,finished,2024-12-26T17:40:28.000Z,403,4,0.231149,NaN,NaN,0.365215,0.358891,...,NaN,NaN,0.656802,0.769397,0.608547,0.994590,0.773307,0.423136,0.548399,0.577731
3,t_initial_floodnet_farseg_single,finished,2024-12-26T17:39:45.000Z,41,0,0.352208,0.352208,0.0,0.451088,0.574218,...,NaN,NaN,0.713493,0.784216,0.683183,0.994038,0.784426,0.543545,0.631728,0.619143
5,t_floodnet_msl_farseg_singlebranch_flood,finished,2024-12-25T12:19:34.000Z,5751,9,0.398562,0.548711,2.0,0.512243,0.558190,...,NaN,NaN,0.195401,0.492577,0.068040,0.952529,0.401402,0.170873,0.504474,0.016100
6,t_initial_floodnet_farseg_singlebranch_flood,finished,2024-12-25T12:12:19.000Z,72,0,0.330023,0.330023,0.0,0.398470,0.280428,...,NaN,NaN,0.656362,0.788808,0.599600,0.987864,0.771432,0.589807,0.703212,0.343311
7,t_floodnet_msl_baseline_singlebranch,finished,2024-12-23T23:32:48.000Z,2764,1,0.250037,NaN,NaN,0.455550,0.601955,...,NaN,NaN,0.608097,0.664975,0.583721,0.990107,0.645262,0.486907,0.488337,0.525090
8,t_initial_floodnet_baseline_singlebranch,finished,2024-12-23T23:28:05.000Z,281,0,0.345982,0.345982,0.0,0.449364,0.584530,...,NaN,NaN,0.685754,0.738435,0.663177,0.991422,0.720276,0.497900,0.719819,0.573137
9,t_floodnet_msl_baseline_singlebranch_flood,finished,2024-12-23T22:32:13.000Z,2237,4,0.615705,0.638861,3.0,0.638898,0.675312,...,NaN,NaN,0.452460,0.715870,0.339569,0.984727,0.660606,0.370826,0.615937,0.126984


In [4]:
(results_initial := results_initial_msl[results_initial_msl["Name"].str.contains("initial")])

,Name,State,Created,Runtime,epoch,challenge_score_target,best_challenge_score_target,best_challenge_score_target_epoch,f1_target,f1_loc_target,...,f1_target_3,f1_target_4,challenge_score_source,f1_loc_source,f1_source_class,f1_source_0,f1_source_1,f1_source_2,f1_source_3,f1_source_4
1,t_initial_floodnet_baseline_singlebranch_flood,finished,2024-12-27T19:42:57.000Z,62,0,0.559746,0.559746,0.0,0.580215,0.651327,...,NaN,NaN,0.599780,0.731908,0.543154,0.983879,0.684317,0.497322,0.664632,0.311290
3,t_initial_floodnet_farseg_single,finished,2024-12-26T17:39:45.000Z,41,0,0.352208,0.352208,0.0,0.451088,0.574218,...,NaN,NaN,0.713493,0.784216,0.683183,0.994038,0.784426,0.543545,0.631728,0.619143
6,t_initial_floodnet_farseg_singlebranch_flood,finished,2024-12-25T12:12:19.000Z,72,0,0.330023,0.330023,0.0,0.398470,0.280428,...,NaN,NaN,0.656362,0.788808,0.599600,0.987864,0.771432,0.589807,0.703212,0.343311
8,t_initial_floodnet_baseline_singlebranch,finished,2024-12-23T23:28:05.000Z,281,0,0.345982,0.345982,0.0,0.449364,0.584530,...,NaN,NaN,0.685754,0.738435,0.663177,0.991422,0.720276,0.497900,0.719819,0.573137


In [5]:
(results_msl := results_initial_msl[results_initial_msl["Name"].str.contains("msl")])

,Name,State,Created,Runtime,epoch,challenge_score_target,best_challenge_score_target,best_challenge_score_target_epoch,f1_target,f1_loc_target,...,f1_target_3,f1_target_4,challenge_score_source,f1_loc_source,f1_source_class,f1_source_0,f1_source_1,f1_source_2,f1_source_3,f1_source_4
2,t_floodnet_msl_farseg_single,finished,2024-12-26T17:40:28.000Z,403,4,0.231149,NaN,NaN,0.365215,0.358891,...,NaN,NaN,0.656802,0.769397,0.608547,0.994590,0.773307,0.423136,0.548399,0.577731
5,t_floodnet_msl_farseg_singlebranch_flood,finished,2024-12-25T12:19:34.000Z,5751,9,0.398562,0.548711,2.0,0.512243,0.558190,...,NaN,NaN,0.195401,0.492577,0.068040,0.952529,0.401402,0.170873,0.504474,0.016100
7,t_floodnet_msl_baseline_singlebranch,finished,2024-12-23T23:32:48.000Z,2764,1,0.250037,NaN,NaN,0.455550,0.601955,...,NaN,NaN,0.608097,0.664975,0.583721,0.990107,0.645262,0.486907,0.488337,0.525090
9,t_floodnet_msl_baseline_singlebranch_flood,finished,2024-12-23T22:32:13.000Z,2237,4,0.615705,0.638861,3.0,0.638898,0.675312,...,NaN,NaN,0.452460,0.715870,0.339569,0.984727,0.660606,0.370826,0.615937,0.126984


In [6]:
(results_finetune := pd.read_csv(RESULTS_PATH / "results_floodnet_finetune.csv").sort_index())

column_mapping = {
    f"f1_{i}_safe": f"f1_target_{i}" for i in range(5)
} | {
    "challenge_score_safe": "best_challenge_score_target",
    "f1_safe": "f1_target",
    "f1_loc_safe": "f1_loc_target",
    "f1_class_safe": "f1_target_class"
}

results_finetune.columns = [column_mapping.get(k, k) for k in results_finetune.columns]
results_finetune

,Name,State,Created,Runtime,epoch,best_challenge_score_target,f1_loc_target,f1_target_class,f1_target,f1_target_0,...,precision_1_safe,precision_2_safe,precision_3_safe,precision_4_safe,recall_safe,recall_0_safe,recall_1_safe,recall_2_safe,recall_3_safe,recall_4_safe
0,t_finetune_floodnet_baseline_singlebranch,finished,2024-12-26T20:13:14.000Z,208,9,0.878453,0.866574,0.883544,0.889341,0.985268,...,0.837728,0.852780,NaN,NaN,0.888089,0.986429,0.783195,0.894642,NaN,NaN
1,t_finetune_floodnet_farseg_single,finished,2024-12-26T20:07:29.000Z,297,9,0.875163,0.869303,0.877675,0.883570,0.985582,...,0.823385,0.860655,NaN,NaN,0.877823,0.986830,0.803603,0.843037,NaN,NaN
2,t_finetune_floodnet_farseg_singlebranch_flood,finished,2024-12-26T19:53:27.000Z,288,9,0.909947,0.905912,0.911677,0.914651,0.989611,...,0.900438,0.863958,NaN,NaN,0.912546,0.990769,0.844925,0.901945,NaN,NaN
3,t_finetune_floodnet_baseline_singlebranch_flood,finished,2024-12-26T17:56:20.000Z,192,9,0.882680,0.871791,0.887347,0.892639,0.985940,...,0.859828,0.848651,NaN,NaN,0.889513,0.987767,0.780890,0.899881,NaN,NaN


In [7]:
def attach_index(df, name_pattern):
    name = df["Name"]
    match = name.str.extract(name_pattern)
    match.columns = ["pass", "model", "pretrained"]
    match["pretrained"] = match.apply(lambda row: "flood" if row["pretrained"] == "flood" else "xbd", axis=1)
    idx = pd.MultiIndex.from_frame(match)
    return df.set_index(idx)
pattern = r"^t_.*?((?:initial|msl|finetune)).*?((?:farseg|baseline)).*?((?:flood)|(?:))$"

cat = pd.concat([
    attach_index(results_initial, pattern),
    attach_index(results_msl, pattern),
    attach_index(results_finetune, pattern),
]).drop(["State", "Created", "Runtime"], axis=1)


fixes = [
    ("msl", "baseline", "xbd", 0.25004),
    ("msl", "farseg", "xbd", 0.23206),
]

for pass_, model, pretrained, best_score in fixes:
    cat.at[(pass_, model, pretrained), "best_challenge_score_target"] = best_score

cat.sort_index()

Name  \
pass     model    pretrained                                                    
finetune baseline flood       t_finetune_floodnet_baseline_singlebranch_flood   
                  xbd               t_finetune_floodnet_baseline_singlebranch   
         farseg   flood         t_finetune_floodnet_farseg_singlebranch_flood   
                  xbd                       t_finetune_floodnet_farseg_single   
initial  baseline flood        t_initial_floodnet_baseline_singlebranch_flood   
                  xbd                t_initial_floodnet_baseline_singlebranch   
         farseg   flood          t_initial_floodnet_farseg_singlebranch_flood   
                  xbd                        t_initial_floodnet_farseg_single   
msl      baseline flood            t_floodnet_msl_baseline_singlebranch_flood   
                  xbd                    t_floodnet_msl_baseline_singlebranch   
         farseg   flood              t_floodnet_msl_farseg_singlebranch_flood   
                  xbd                            t_floodnet_msl_farseg_single   

                              epoch  challenge_score_target  \
pass     model    pretrained                                  
finetune baseline flood           9                     NaN   
                  xbd             9                     NaN   
         farseg   flood           9                     NaN   
                  xbd             9                     NaN   
initial  baseline flood           0                0.559746   
                  xbd             0                0.345982   
         farseg   flood           0                0.330023   
                  xbd             0                0.352208   
msl      baseline flood           4                0.615705   
                  xbd             1                0.250037   
         farseg   flood           9                0.398562   
                  xbd             4                0.231149   

                              best_challenge_score_target  \
pass     model    pretrained                                
finetune baseline flood                          0.882680   
                  xbd                            0.878453   
         farseg   flood                          0.909947   
                  xbd                            0.875163   
initial  baseline flood                          0.559746   
                  xbd                            0.345982   
         farseg   flood                          0.330023   
                  xbd                            0.352208   
msl      baseline flood                          0.638861   
                  xbd                            0.250040   
         farseg   flood                          0.548711   
                  xbd                            0.232060   

                              best_challenge_score_target_epoch  f1_target  \
pass     model    pretrained                                                 
finetune baseline flood                                     NaN   0.892639   
                  xbd                                       NaN   0.889341   
         farseg   flood                                     NaN   0.914651   
                  xbd                                       NaN   0.883570   
initial  baseline flood                                     0.0   0.580215   
                  xbd                                       0.0   0.449364   
         farseg   flood                                     0.0   0.398470   
                  xbd                                       0.0   0.451088   
msl      baseline flood                                     3.0   0.638898   
                  xbd                                       NaN   0.455550   
         farseg   flood                                     2.0   0.512243   
                  xbd                                       NaN   0.365215   

                              f1_loc_target  f1_target_class  f1_target_0  \
pass     model    pretrained  

In [8]:
_df = cat[["best_challenge_score_target"]].sort_index()
_df = pd.concat([_df, cat[
    ["f1_target", "f1_loc_target", "f1_target_class"]
    + [f"f1_target_{i}" for i in range(3)]
    ]], axis=1)
_df

df = pd.concat([
    _df.loc["initial"].add_prefix("ini_"),
    _df.loc["msl"].add_prefix("msl_"),
    _df.loc["finetune"].add_prefix("ft_"),
], axis=1)
df


ini_best_challenge_score_target  ini_f1_target  \
model    pretrained                                                   
baseline flood                              0.559746       0.580215   
         xbd                                0.345982       0.449364   
farseg   flood                              0.330023       0.398470   
         xbd                                0.352208       0.451088   

                     ini_f1_loc_target  ini_f1_target_class  ini_f1_target_0  \
model    pretrained                                                            
baseline flood                0.651327             0.485963         0.957921   
         xbd                  0.584530             0.177787         0.956251   
farseg   flood                0.280428             0.133401         0.949505   
         xbd                  0.574218             0.167062         0.958993   

                     ini_f1_target_1  ini_f1_target_2  \
model    pretrained                                     
baseline flood              0.415201         0.367524   
         xbd                0.312666         0.079176   
farseg   flood              0.062575         0.183329   
         xbd                0.321847         0.072424   

                     msl_best_challenge_score_target  msl_f1_target  \
model    pretrained                                                   
baseline flood                              0.638861       0.638898   
         xbd                                0.250040       0.455550   
farseg   flood                              0.548711       0.512243   
         xbd                                0.232060       0.365215   

                     msl_f1_loc_target  ...  msl_f1_target_0  msl_f1_target_1  \
model    pretrained                     ...                                     
baseline flood                0.675312  ...         0.963409         0.490686   
         xbd                  0.601955  ...         0.958842         0.381409   
farseg   flood                0.558190  ...         0.908460         0.425016   
         xbd                  0.358891  ...         0.953202         0.142286   

                     msl_f1_target_2  ft_best_challenge_score_target  \
model    pretrained                                                    
baseline flood              0.462599                        0.882680   
         xbd                0.026398                        0.878453   
farseg   flood              0.203254                        0.909947   
         xbd                0.000157                        0.875163   

                     ft_f1_target  ft_f1_loc_target  ft_f1_target_class  \
model    pretrained                                                       
baseline flood           0.892639          0.871791            0.887347   
         xbd             0.889341          0.866574            0.883544   
farseg   flood           0.914651          0.905912            0.911677   
         xbd             0.883570          0.869303            0.877675   

                     ft_f1_target_0  ft_f1_target_1  ft_f1_target_2  
model    pretrained                                                  
baseline flood             0.985940        0.818460        0.873516  
         xbd               0.985268        0.809544        0.873210  
farseg   flood             0.989611        0.871799        0.882543  
         xbd               0.985582        0.813373        0.851755  

[4 rows x 21 columns]

In [9]:
df1 = df.reset_index()
query = f"""
SELECT
    model, pretrained,
    ini_best_challenge_score_target "ini_score",
    msl_best_challenge_score_target "msl_score",
    ft_best_challenge_score_target "ft_score",
    msl_best_challenge_score_target - ini_best_challenge_score_target "msl_change_abs",
    (msl_best_challenge_score_target - ini_best_challenge_score_target) * 100 / ini_best_challenge_score_target "msl_change_percent",
    ft_best_challenge_score_target - ini_best_challenge_score_target "ft_change_abs",
    (ft_best_challenge_score_target - ini_best_challenge_score_target) * 100 / ini_best_challenge_score_target "ft_change_percent"
FROM df1
ORDER BY pretrained, model
"""
display(HTML(duckdb.query(query).df().to_html()))
for i in range(3):
    query = f"""
    SELECT
        model, pretrained,
        ini_f1_target_{i} "ini_f1_{i}",
        msl_f1_target_{i} "msl_f1_{i}",
        ft_f1_target_{i} "ft_f1_{i}",
        msl_f1_target_{i} - ini_f1_target_{i} "msl_f1_{i}_change_abs",
        (msl_f1_target_{i} - ini_f1_target_{i}) * 100 / ini_f1_target_{i} "msl_f1_{i}_change_percent",
        ft_f1_target_{i} - ini_f1_target_{i} "f1_f1_{i}_change_abs",
        (ft_f1_target_{i} - ini_f1_target_{i}) * 100 / ini_f1_target_{i} "ft_f1_{i}_change_percent"
    FROM df1
    ORDER BY pretrained, model
    """
    print(f"################## CLASS {i} ##################")
    display(HTML(duckdb.query(query).df().to_html()))

,model,pretrained,ini_score,msl_score,ft_score,msl_change_abs,msl_change_percent,ft_change_abs,ft_change_percent
0,baseline,flood,0.559746,0.638861,0.882680,0.079115,14.134057,0.322934,57.692868
1,farseg,flood,0.330023,0.548711,0.909947,0.218689,66.264661,0.579924,175.722514
2,baseline,xbd,0.345982,0.250040,0.878453,-0.095942,-27.730273,0.532471,153.901615
3,farseg,xbd,0.352208,0.232060,0.875163,-0.120148,-34.112806,0.522955,148.479046


################## CLASS 0 ##################


,model,pretrained,ini_f1_0,msl_f1_0,ft_f1_0,msl_f1_0_change_abs,msl_f1_0_change_percent,f1_f1_0_change_abs,ft_f1_0_change_percent
0,baseline,flood,0.957921,0.963409,0.985940,0.005489,0.573005,0.028020,2.925045
1,farseg,flood,0.949505,0.908460,0.989611,-0.041046,-4.322860,0.040106,4.223840
2,baseline,xbd,0.956251,0.958842,0.985268,0.002591,0.271005,0.029018,3.034540
3,farseg,xbd,0.958993,0.953202,0.985582,-0.005792,-0.603925,0.026589,2.772605


################## CLASS 1 ##################


,model,pretrained,ini_f1_1,msl_f1_1,ft_f1_1,msl_f1_1_change_abs,msl_f1_1_change_percent,f1_f1_1_change_abs,ft_f1_1_change_percent
0,baseline,flood,0.415201,0.490686,0.818460,0.075486,18.180516,0.403260,97.124037
1,farseg,flood,0.062575,0.425016,0.871799,0.362441,579.209006,0.809224,1293.202447
2,baseline,xbd,0.312666,0.381409,0.809544,0.068743,21.986136,0.496878,158.916435
3,farseg,xbd,0.321847,0.142286,0.813373,-0.179561,-55.790703,0.491527,152.720631


################## CLASS 2 ##################


,model,pretrained,ini_f1_2,msl_f1_2,ft_f1_2,msl_f1_2_change_abs,msl_f1_2_change_percent,f1_f1_2_change_abs,ft_f1_2_change_percent
0,baseline,flood,0.367524,0.462599,0.873516,0.095075,25.869119,0.505992,137.675866
1,farseg,flood,0.183329,0.203254,0.882543,0.019925,10.868458,0.699214,381.398383
2,baseline,xbd,0.079176,0.026398,0.873210,-0.052779,-66.659834,0.794033,1002.866469
3,farseg,xbd,0.072424,0.000157,0.851755,-0.072267,-99.782809,0.779331,1076.067320
